In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.schema import BaseOutputParser
from langchain.prompts import ChatPromptTemplate,PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate

chat= ChatOpenAI(temperature=0.1,
                 streaming=True,
                 callbacks=[StreamingStdOutCallbackHandler(),],
                 )

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]


example_prompt = PromptTemplate.from_template("Human : {question}\nAI :{answer}")

prompt= FewShotPromptTemplate(
    example_prompt= example_prompt,
    examples= examples,
    suffix= "Human : what do you know about {country}?",
    input_variables= ["country"]
)

prompt.format(country= "Germany")

chain = prompt | chat

chain.invoke({
    "country" : "Turkey"
})

AI :
I know this:
Capital: Ankara
Language: Turkish
Food: Kebab and Baklava
Currency: Turkish Lira

AIMessageChunk(content='AI :\nI know this:\nCapital: Ankara\nLanguage: Turkish\nFood: Kebab and Baklava\nCurrency: Turkish Lira')